In [ ]:
# create combined stopwords file
# def append_file(source_file, destination_file):
#     with open(source_file, 'r') as source, open(destination_file, 'a') as destination:
#         for line in source:
#             destination.write(line)

# # Example usage
# source_file = 'stopwords\StopWords_Currencies.txt'  # Replace with the filename of the source text file
# destination_file = 'stopwords\StopWords.txt'  # Replace with the filename of the destination text file

# append_file(source_file, destination_file)

In [3]:
import pandas as pd

In [4]:
# create a stop word list
stop_list = []
with open('stopwords\StopWords.txt', 'r') as stop:
    for i in stop:
        stop_list.append(i.strip())

len(stop_list)

14104

In [5]:
# positive words list
pos_list = []
with open('positive-words.txt', 'r') as pos:
    for i in pos:
        pos_list.append(i.strip())

In [6]:
# read the output file
output = pd.read_csv("Output Data Structure.xlsx - Sheet1.csv")

In [7]:
neg_list = []
with open('negative-words.txt', 'r', encoding="ISO-8859-1") as neg:
    for j in neg:
        neg_list.append(j.strip())

In [8]:
# read the 37.txt for sentiment analysis
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Download the necessary NLTK data (only required for the first time)
# nltk.download('punkt')

In [9]:
import string

In [10]:
def tokenize_text(filename):
    with open(filename, 'r') as file:
        text = file.read()

    # Tokenize the text into words
    word_tokens = word_tokenize(text)

    # Tokenize the text into sentences
    sentence_tokens = sent_tokenize(text)

    return word_tokens, sentence_tokens

In [13]:
# 37.txt file usage
output_words = []
output_sen = []

for i in range(37,151):
    filename = f'content_files\{i}.txt'  # Replace with the filename of the text file
    word_tokens, sentence_tokens = tokenize_text(filename)
    output_words.append(word_tokens)
    output_sen.append(sentence_tokens)

In [15]:
len(output_words), len(output_sen)

(114, 114)

In [28]:
# add the both tokens list in output df
output['WORDS_TOKEN'] = output_words
output['SEN_TOKEN'] = output_sen

In [29]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            114 non-null    int64  
 1   URL                               114 non-null    object 
 2   POSITIVE SCORE                    0 non-null      float64
 3   NEGATIVE SCORE                    0 non-null      float64
 4   POLARITY SCORE                    0 non-null      float64
 5   SUBJECTIVITY SCORE                0 non-null      float64
 6   AVG SENTENCE LENGTH               0 non-null      float64
 7   PERCENTAGE OF COMPLEX WORDS       0 non-null      float64
 8   FOG INDEX                         0 non-null      float64
 9   AVG NUMBER OF WORDS PER SENTENCE  0 non-null      float64
 10  COMPLEX WORD COUNT                0 non-null      float64
 11  WORD COUNT                        0 non-null      float64
 12  SYLLABLE

In [24]:
# filter the stop words from the word tokens
# final_words = [word for word in word_tokens if word not in stop_list]
# len(final_words)

In [11]:
# filter the punctuation
punc = string.punctuation + '""''“”’~'
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~""“”’~'

In [25]:
# clean_words = [word for word in final_words if word not in punc]
# len(clean_words)

In [31]:
# write a positive score func
def positive_score(words):
    final_words = [word for word in words if word not in stop_list]
    count = 0
    for word in final_words:
        if word in pos_list:
            count += 1
    return count

In [32]:
# add the positive score to the each rows of df
output['POSITIVE SCORE'] = output['WORDS_TOKEN'].map(lambda x: positive_score(x))

In [36]:
# write a negative score func
def negative_score(words):
    final_words = [word for word in words if word not in stop_list]
    count = 0
    for word in final_words:
        if word in neg_list:
            count += -1
    return count*-1

In [37]:
# add the neg score to the each rows of df
output['NEGATIVE SCORE'] = output['WORDS_TOKEN'].map(lambda x: negative_score(x))

In [39]:
# polarity score
def polarity_score(pos_score, neg_score):
    polarity = (pos_score - neg_score)/((pos_score + neg_score) + 0.000001)
    return polarity

In [40]:
# calculate polarity score for each and every rows
output['POLARITY SCORE'] = (output['POSITIVE SCORE'] - output['NEGATIVE SCORE'])/((output['POSITIVE SCORE'] + output['NEGATIVE SCORE']) + 0.000001)

In [43]:
# subjectiviy score
def subjectivity_score(pos_score, neg_score, words):
    final_words = [word for word in words if word not in stop_list]
    clean_words = [word for word in final_words if word not in punc]
    sub = (pos_score + neg_score)/((len(clean_words)) + 0.000001)
    return sub

In [44]:
# apply subjectivity function to df
output['SUBJECTIVITY SCORE'] = output.apply(lambda row: subjectivity_score(row['POSITIVE SCORE'], 
                                                                        row['NEGATIVE SCORE'],
                                                                        row['WORDS_TOKEN']), axis=1)

In [49]:
# find the avg word per sentence
def avg_word_len(word_token, sen_token):
    words = [word for word in word_token if word not in punc]
    avg_len = len(words)/len(sen_token)
    return avg_len

In [66]:
output['SEN_TOKEN'] = output['SEN_TOKEN'].apply(lambda x: [1] if len(x) == 0 else x)

In [68]:
# apply avg world len to the output df
output['AVG NUMBER OF WORDS PER SENTENCE'] = output.apply(lambda row: avg_word_len(row['WORDS_TOKEN'],
                                                                                    row['SEN_TOKEN']), axis=1)

In [70]:
# find the avg sentence length (which includes punctuation)
def avg_sen(word_token, sen_token):
    avg_words = len(word_token)/len(sen_token)
    return avg_words

In [71]:
# apply thg avg_sen to the df
output['AVG SENTENCE LENGTH'] = output.apply(lambda row: avg_sen(row['WORDS_TOKEN'],
                                                                row['SEN_TOKEN']), axis=1)

In [73]:
# word count (after cleaning)
def word_count(word_token):
    final_word = [word for word in word_token if word not in stop_list]
    clean_word = [word for word in final_word if word not in punc]
    return len(clean_word)

In [75]:
# applt the word_count func to the df
output['WORD COUNT'] = output.apply(lambda row: word_count(row['WORDS_TOKEN']), axis=1)

In [ ]:
# pip install pyphen

In [77]:
# syllable count 
import pyphen

def count_syllables(word):
    dic = pyphen.Pyphen(lang='en_US')
    syllables = dic.inserted(word).count('-') + 1
    return syllables

def count_syllables_corpus(corpus):
    syllable_count = 0
    for word in corpus:
        syllables = count_syllables(word)
        syllable_count += syllables
    return syllable_count

In [85]:
# syllable count per word
def syllable_count(words_list):
    final_word = [str(word) for word in words_list if word not in stop_list]
    clean_word = [str(word) for word in final_word if word not in punc]
    syble_count = count_syllables_corpus(clean_word)
    syble_word = syble_count/len(clean_word)
    return syble_word

In [81]:
output['WORDS_TOKEN'] = output['WORDS_TOKEN'].apply(lambda x: [1] if len(x) == 0 else x)

In [86]:
# apply the syllable count func to the df
output['SYLLABLE PER WORD'] = output.apply(lambda row: syllable_count(row['WORDS_TOKEN']), axis=1)

In [92]:
# complex word count
def com_syble(corpus):
    compelx_words = 0
    for word in corpus:
        syllables = count_syllables(word)
        if syllables > 2:
            compelx_words += 1      
    return compelx_words
    

def complex_word(words):
    final_word = [str(word) for word in words if word not in stop_list]
    clean_word = [str(word) for word in final_word if word not in punc]
    com_count = com_syble(clean_word)
    return com_count

In [93]:
# apply complex word count func to the df
output['COMPLEX WORD COUNT'] = output.apply(lambda row: complex_word(row['WORDS_TOKEN']), axis=1)

In [97]:
# percentage of complex words
def per_compelx_word(words):
    final_word = [str(word) for word in words if word not in stop_list]
    clean_word = [str(word) for word in final_word if word not in punc]
    com_count = com_syble(clean_word)
    percentage = com_count/len(clean_word)
    return round(round(percentage, 3)*100,3)

In [98]:
# apply the complex words percentage func to the df
output['PERCENTAGE OF COMPLEX WORDS'] = output.apply(lambda row: per_compelx_word(row['WORDS_TOKEN']), axis=1)

In [104]:
# fog index
def fog_index(words_lis, sen_lis):
    len_words = len(words_lis)/len(sen_lis)
    
    final_word = [str(word) for word in words_lis if word not in stop_list]
    clean_word = [str(word) for word in final_word if word not in punc]
    com_count = com_syble(clean_word)
    percentage = com_count/len(clean_word)
    fog = 0.4*(len_words + percentage)
    return fog

In [105]:
# apply the fog index func to the df
output['FOG INDEX'] = output.apply(lambda row: fog_index(row['WORDS_TOKEN'],row['SEN_TOKEN']), axis=1)

In [107]:
# let's count personal pronouns in text words
import re

def extract_personal_pronouns(words):
    pattern = r'\b(I|me|my|mine|we|us|our|ours|you|your|yours|he|him|his|she|her|hers|it|its|they|them|their|theirs|They|You|Our|My|He|She|We)\b'
    personal_pronouns = re.findall(pattern, ' '.join(words))
    return len(personal_pronouns)

In [110]:
output['WORDS_TOKEN'] = output['WORDS_TOKEN'].apply(lambda x: ['a','b'] if len(x) == 1 else x)

In [111]:
# apply personal pronouns func to the df
output['PERSONAL PRONOUNS'] = output.apply(lambda row: extract_personal_pronouns(row['WORDS_TOKEN']), axis=1)

In [116]:
# avg word length
def avg_word_char_len(words_list):
    clean_word = [word for word in words_list if word not in punc]
    len_char = "".join(clean_word)
    count_len = len(len_char)
    count_word = len(clean_word)
    avg = count_len/count_word
    return round(avg, 3)

In [117]:
# apply avg word lenght to the df
output['AVG WORD LENGTH'] = output.apply(lambda row: avg_word_char_len(row['WORDS_TOKEN']), axis=1)

In [121]:
# remove non-useful columns
df_output = output.drop(['WORDS_TOKEN','SEN TOKEN','SEN_TOKEN'], axis=1)

In [125]:
# Save the DataFrame as an Excel file
df_output.to_excel('output.xlsx', index=False)